In [44]:
import torch
import torch.nn as nn
import torch.optim as optim

In [45]:
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))

In [46]:
word2index = {tkn:i for i,tkn in enumerate(vocab, 1)}
word2index['<unk>'] = 0
index2word = {v:k for k,v in word2index.items()}

In [47]:
def build_data(sentence, word2index):
    encoded = [word2index[tkn] for tkn in sentence]
    input_seq, label_seq = encoded[:-1], encoded[1:]
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq

In [48]:
X, Y = build_data(sentence, word2index)

In [52]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first = True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings = vocab_size, embedding_dim = input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first = batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x):
        output = self.embedding_layer(x)
        output, hidden = self.rnn_layer(output)
        output = self.linear(output)
        return output.view(-1, output.size(2))

In [58]:
vocab_size = len(word2index)
input_size = 5
hidden_size = 20

model = Net(vocab_size, input_size, hidden_size, batch_first = True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

In [55]:
output = model(X)
decode = lambda y: [index2word.get(x) for x in y]

In [59]:
for step in range(250):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0308 
Repeat Repeat the <unk> Repeat the <unk>

[41/201] 1.3739 
Repeat for the best medicine for memory

[81/201] 0.7711 
Repeat is the best medicine for memory

[121/201] 0.4252 
Repeat is the best medicine for memory

[161/201] 0.2370 
Repeat is the best medicine for memory

[201/201] 0.1373 
Repeat is the best medicine for memory

[241/201] 0.0880 
Repeat is the best medicine for memory

